In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import copy
import itertools
from tqdm import tqdm
from textblob import TextBlob
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
import networkx as nx
import matplotlib.pyplot as plt
from scipy import spatial
%matplotlib inline
import math
import random 
from scipy.spatial import distance

In [ ]:
Univ2_data=pd.read_csv('Univ2_dataset.csv',encoding = "ISO-8859-1")

In [ ]:
Univ2_data.info()

REMOVE STATS 266,245

In [ ]:
#removing two not required rows
Univ2_data.drop([20,23],inplace=True)

In [ ]:
#removing all the department number starting with 1
Univ2_data.drop(Univ2_data[Univ2_data['DepartmentNumber'].str.contains('^1')].index, inplace = True) 

In [ ]:
#checking for the department number if all the numbers starting with 1 are not present
Univ2_data['DepartmentNumber'].str.contains('^1', case=False)

In [ ]:
#removing the duplicates rows based on course number
Univ2_data.drop_duplicates(subset=['course_number'],inplace=True)

In [ ]:
#to reset the index
Univ2_data.reset_index(drop=True,inplace=True)

In [ ]:
Univ2_data.head()

In [ ]:
Univ2_data[Univ2_data['course_number']=='STATS 306B'].index

In [ ]:
#creating the six lists with courses denoting six different areas required to complete the degree
data_science_math_list=['STATS 200','STATS 300A','STATS 203','STATS 305A','STATS 315A','CME 302','CME 308']
data_science_experimentative_elective=['STATS 263','ECON 271','MS&E 327']
data_science_software_list=['CME 212','CME 213','CME 305','CME 307','CME 323','CME 364A','CS 246']
data_science_machine_learning_list=['STATS 231','STATS 315B','CS 221','CS 224N','CS 230','CS 231N','CS 234','CS 236']
data_science_practical_list=['STATS 299','STATS 390']
data_science_elective_list=['STATS 306B','CS 228','CS 229','CME 211','MATH 21','STATS 305','ECON 270',
 'CME 200','STATS 204']

In [ ]:
#creating the prereq dict by checking from univ-2 website
prereq_dict_3={'CME 212': [['CME 200', 'CME 211']],
 'CME 213': [],
 'CME 302': [],
 'CME 305': [],
 'CME 308': [],
 'CME 323': [],
 'CME 364A': [],
 'CS 221': [],
 'CS 224N': ['CS 221', 'CS 229'],
 'CS 230': [],
 'CS 231N': [['CS 229', 'MATH 21']],
 'CS 234': ['CS 229'],
 'CS 236': ['CS 221', 'CS 228', 'CS 229', 'CS 230'],
 'CS 246': [],
 'ECON 271': ['ECON 270'],
 'MS&E 327': [],
 'STATS 200': [],
 'STATS 203': [],
 'STATS 204': [],
 'STATS 231': [],
 'STATS 299': [],
 'STATS 305A': [],
 'STATS 315A': ['STATS 305','STATS 306B'],
 'STATS 315B': [],
 'STATS 390': []}

In [ ]:
#Converting the format of title column in the dataset to string format
Univ2_data['Title']=Univ2_data['Title'].apply(str)

#cleansing of data
def clean_text(text):
    #Convert to lowercase to maintain consistency
    text = text.lower()
    return text

Univ2_data['Title']=Univ2_data['Title'].apply(clean_text)

In [ ]:
#tokenizing the text
def getting_nouns(text):
  text_blob_object=TextBlob(text)

  return text_blob_object.words


Univ2_data['words']=Univ2_data['Title'].apply(getting_nouns)

In [ ]:
#removing stopwords using nltk library

Univ2_data['words'] = Univ2_data['words'].apply(lambda x:[item for item in x if item not in stop])

#converting words column from the dataset into a single list
list_of_topics=Univ2_data['words'].tolist()

#converting into a single list
merged = list(itertools.chain.from_iterable(list_of_topics))

#removing the duplicates from the list
top_100_topics=list(dict.fromkeys(merged))


In [ ]:
#Topic areas
print(*top_100_topics,sep=",")

In [ ]:
#making units dictionary to track the course and its numbers of units 
course_units_dict={}
for i in range(len(Univ2_data['course_number'])):
  course_units_dict[Univ2_data['course_number'][i]]=Univ2_data['UnitsMin'][i]

course_units_dict['ECON 271']=3
course_units_dict['STATS 300A']=3
course_units_dict['STATS 390']=1
course_units_dict['STATS 299']=2

In [ ]:
course_units_dict

In [ ]:
#Update function for feature vector update at a node

def update_feature_vector(feature_vector,topic_vector,course_list):
  for i in course_list[0]:
    for j in range(len(topic_vector)):
      if i==topic_vector[j]:
         feature_vector[j]=1
  return feature_vector 

#Creating the feature dictionary with course as key and course topics as values
feature_ds_dict={}
course_list_ds=list(Univ2_data['course_number'])
for i in tqdm(course_list_ds):
  temp = list(Univ2_data[Univ2_data['course_number']== i ]['words'])
  feature_vector_ds =[0]*len(top_100_topics)
  feature_vector_ds = update_feature_vector(feature_vector_ds,top_100_topics,temp)
  feature_ds_dict[i]=feature_vector_ds

In [ ]:
#printing the dictionary
for i,j in feature_ds_dict.items():
  print('{} : {}'.format(i,j))

In [ ]:
#to compile one dictinary to distinguish between six different areas.
data_science_dict={}
for i in range(len(course_list_ds)):
  key=course_list_ds[i]
  if key in data_science_math_list:
    data_science_dict[key]=[1,0,0,0,0,0]
  elif key in data_science_experimentative_elective:
    data_science_dict[key]=[0,1,0,0,0,0]
  elif key in data_science_software_list:
    data_science_dict[key]=[0,0,1,0,0,0]
  elif key in data_science_machine_learning_list:
    data_science_dict[key]=[0,0,0,1,0,0]
  elif key in data_science_practical_list:
    data_science_dict[key]=[0,0,0,0,1,0]
  else:
    data_science_dict[key]=[0,0,0,0,0,1]

In [ ]:
data_science_dict

In [ ]:
#creating prereq dictionary where course is key and prereqs are values
def prereq_dict(data_dict,course_list):
  course_dict={}
  for i in range(len(course_list)):
    key=course_list[i]
    if key in data_dict:
      course_dict[key]=data_dict.get(key)
    else:
      course_dict[key]=[]
  return course_dict

In [ ]:
data_science_prereq_dict=prereq_dict(prereq_dict_3,course_list_ds)

In [ ]:
for i,j in data_science_prereq_dict.items():
  print('{} : {}'.format(i,j))

In [ ]:
data_science_prereq_dict['STATS 305A']=['STATS 200']

In [ ]:
#create the graph
edges=[]
for course in course_list_ds:
  CourseId=course_list_ds[:]
  CourseId.remove(course)
  for i in CourseId:
    edges.append((i,course))
 
for course in course_list_ds:
   edges.append(('initial' , course))
       
G = nx.DiGraph()
G.add_edges_from(edges) 
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G,pos, cmap=plt.get_cmap('jet'),node_size = 1000,node_color="lightblue")
nx.draw_networkx_edges(G,pos,edge_color='b', edge_cmap=plt.cm.Blues,arrows=True,arrowstyle="->",arrowsize=10)
nx.draw_networkx_labels(G,pos)

plt.show()

In [ ]:
nodes={}
i=0
for i,node in enumerate(G.nodes()):
  nodes[i]=node

In [ ]:
def getKeysByValue(dictOfElements, valueToFind):
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            key=item[0]
    return  key

In [ ]:
actionVectors={}
for edge in edges:
  actionVectors[edge]=data_science_dict[edge[1]]

In [ ]:
#Function to update Q Value
def findKey(course):
  return list(nodes.keys())[list(nodes.values()).index(course)]

In [ ]:
'''Bitwise addition of two lists '''

def addBinary(l1,l2):
  '''Adds two binary lists'''
  if (len(l1) != len(l2)):
    return "Lists must be same length"
  l3 = []
  for i in range(len(l1)):
    if(l1[i]!=l2[i]):
      l3.append(l1[i]+l2[i])
    elif(l1[i]>l2[i]):
      l3.append(l1[i])
    else:
      l3.append(l2[i])
  return l3

In [ ]:
import numpy
import statistics 

def levenshteinDistanceDP(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))


    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

In [ ]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

In [ ]:
''' Returns the maximum number of consecutive numbers that occured in a list '''

def consecutive(list_of_element,element):
    
    list_of_index=get_index_positions(list_of_element, element)
    
    occurance=[]
    count=1
    i=0
    if (len( list_of_index)>=2):
      while i < (len(list_of_index)-1):
        if (list_of_index[i+1] == list_of_index[i]+1):
          count+=1
          i+=1
        else:
          occurance.append(count)
          i+=1
          count=1
      occurance.append(count)

      maximum=max(occurance)
      length=len(list_of_element)
      sum_list=sum(occurance)
      occur=(sum_list * maximum) / length
    elif (list_of_index==[]):
      occur=0
    else:
      occur=list_of_element[list_of_index[0]]


    return occur

In [ ]:
def prereq(List, course):

  if (data_science_prereq_dict[course]!=[]):
    if (type(data_science_prereq_dict[course][0])==list):
      for j in data_science_prereq_dict[course][0]:
        if ((j in List)==False or (len(List)- get_index_positions(List,j)[0]) <2 ):
          return 0 
      return 1
    else:
      for j in data_science_prereq_dict[course]:
        if ((j in List)==True  and (len(List)- get_index_positions(List,j)[0])>2 ):
          return 1
      return 0

  else:
    return 1

In [ ]:
def prereq1(List, number):

  if (data_science_prereq_dict[nodes[number]]!=[]):
    if (type(data_science_prereq_dict[nodes[number]][0])==list):
      for j in data_science_prereq_dict[nodes[number]][0]:
        if ((findKey(j) in List)==False):
          return 0 
      return 1
    else:
      for j in data_science_prereq_dict[nodes[number]]:
        if ((findKey(j) in List)==True):
          return 1
      return 0

  else:
    return 1

In [ ]:
dataScience={}
for key in actionVectors:
  if (actionVectors[key]==[1,0,0,0,0,0]):
    dataScience[key]="math"
  elif (actionVectors[key]==[0,1,0,0,0,0]):
    dataScience[key]="expelective"
  elif (actionVectors[key]==[0,0,1,0,0,0]):
    dataScience[key]="software"
  elif (actionVectors[key]==[0,0,0,1,0,0]):
    dataScience[key]="machinelearning"
  elif (actionVectors[key]==[0,0,0,0,1,0]):
    dataScience[key]="practicalComponent"
  else:
    dataScience[key]="elective"

In [ ]:
#Class that defines the Enviornment
#Our enviornment follows a graph based model of a data set

class Env:
  def __init__(self,input_graph,goal_vect,action_vect,startState):
    self.input_graph = input_graph #NetworkX graph
    self.goal_vect = goal_vect #Goal Vector/Features
    self.action_vect = action_vect # Action Vectors Dictionary
    self.startState = startState #Integer representing the starting state 
   

  
  def step(self,state1,action1,state1Vect, stateList,w1,w2,w3,w4,w5,w6,delta,beta,units_vect,course_vector): # to take the action 
    done = False
    action1Vect = self.action_vect[action1][:]
  
    state1Vect.append(dataScience[action1])
    state2Vect = state1Vect[:]
  
    if (sum(units_vect)>=45): #Have same num of courses as the goal vector, so terminate
      done = True
    
    #Calculate reward of the given move
    rewardList=[]
    for i in range(len(self.goal_vect)):
      
      distance=levenshteinDistanceDP(state2Vect[len(state2Vect)-1], self.goal_vect[i][len(state2Vect)-1])
      if distance==0.0:
        stateList[i].append(1)
      else:
        stateList[i].append(0)
      rewardList.append(consecutive(stateList[i],1))

    rewardTotal=0


    if dataScience[action1]=="math":
      rewardTotal= delta * statistics.mean(rewardList) +  beta * w1
    elif dataScience[action1]=="expelective":
      rewardTotal= delta * statistics.mean(rewardList) +  beta * w2
    elif dataScience[action1]=="software":
      rewardTotal= delta * statistics.mean(rewardList) +  beta * w3
    elif dataScience[action1]=="machinelearning":
      rewardTotal= delta * statistics.mean(rewardList) +  beta * w4
    elif dataScience[action1]=="practicalComponent":
      rewardTotal= delta * statistics.mean(rewardList) +  beta * w5
    else:
      rewardTotal= delta * statistics.mean(rewardList) +  beta * w6


    state2 = action1[1]
    # Return second state, reward,done flag
    return state2,state2Vect,rewardTotal,stateList,units_vect,done

 


  def reset (self):
    #This will reset to initial state 
    # We invoke this on each episode 
    return self.startState


In [ ]:
#Define function to choose action given a state, as per policy of our paper
#Policy is applied to the seed state on each iteration

def choose_action(state,stateUTSoFar,topicVector,goalVector,dataScience,possible_states,stateVector, stateList,course_vector,w1,w2,w3,w4,w5,w6,delta,beta,units_vect):
  
  ideal=[1]*73
  possible_actions={}
   # Get list of possible actions 
  for action in dataScience.keys():
    if (action[0] == state and (action[1] in possible_states)== True):
      
      if (all(i == 0 for i in feature_ds_dict[action[1]])):
        continue
      else:
        a=addBinary(topicVector,feature_ds_dict[action[1]])
        b=1 - spatial.distance.cosine(topicVector, ideal)
        c=1 - spatial.distance.cosine(a, ideal)
        if (c-b>0.0025): # This is cosine threshold to choose action
        
          possible_actions[action] = dataScience[action][:]
    else:
      continue
  
  stateL=[]
  utility={}
  math_list=0
  for action in possible_actions:
    #first statevector value is core
    state_vect=stateVector[:]
    
    
    state_vect.append(possible_actions[action])

    if (len(state_vect)<=15):

      List=[]
      
      state_list=copy.deepcopy(stateList)
      for i in range(len(goalVector)):
        distance=levenshteinDistanceDP(state_vect[len(state_vect)-1], goalVector[i][len(state_vect)-1])
        
        if distance==0:
          state_list[i].append(1)
        else:
          state_list[i].append(0)
        
        List.append(consecutive(state_list[i],1))

      currStateUT=0
      
      if dataScience[action]=="math":
        currStateUT= (delta * statistics.mean(List) + beta * w1) * prereq(course_vector,action[1])
      elif dataScience[action]=="expelective":
        currStateUT= (delta * statistics.mean(List) + beta * w2) * prereq(course_vector,action[1])
      elif dataScience[action]=="software":
        currStateUT= (delta * statistics.mean(List) + beta * w3) * prereq(course_vector,action[1])
      elif dataScience[action]=="machinelearning":
        currStateUT= (delta * statistics.mean(List) + beta * w4) * prereq(course_vector,action[1])
      elif dataScience[action]=="practicalComponent":
        currStateUT= (delta * statistics.mean(List) + beta * w5) * prereq(course_vector,action[1])
      else:
        currStateUT= (delta * statistics.mean(List) + beta * w6) * prereq(course_vector,action[1])

      
      sess_utility  = currStateUT + stateUTSoFar 
      utility[action]=sess_utility
      
      
    else:
      
      break

  ''''for i,j in utility.items():
    print('{} : {}'.format(i,j))'''
  candidate=[]
  if (utility!={}):
    maximum=max(utility.values())
    for i in utility:
      if (utility[i]==maximum):
        candidate.append(i)

    chosen_action = random.choice(candidate)
    stateUTSoFar=utility[chosen_action]
    
  else:
    chosen_action=()
  
  # if chosen_action==():
  #   print (chosen_action)
  # else:
  #   print('chosen action 2:' , chosen_action[1] ,'domain=', dataScience[chosen_action])


  if (chosen_action==()):
      topicVect2=topicVector
  else:
      topicVect2=addBinary(topicVector, feature_ds_dict[chosen_action[1]])  


  return chosen_action , stateUTSoFar , topicVect2,units_vect,course_vector

In [ ]:
#SARSA
def update(state, state2, action, action2,reward,gamma,alpha): 
    action=action[1]
    action2=action2[1]
    predict = Q[findKey(state), findKey(action)] 
    target = reward + gamma * Q[findKey(state2), findKey(action2)] 
    Q[findKey(state), findKey(action)]  = Q[findKey(state), findKey(action)]  + alpha * (target - predict)

In [ ]:
#Initialize parameters (Set as per preferences, we set to ideal valuess)

total_episodes =100

alpha = 0.95 #Learning Rate
gamma = 0.75 # Discount Factor
  
#Initializing the Q-matrix 
numOfStates = G.number_of_nodes() 
Q = np.matrix(np.zeros(shape =(numOfStates, numOfStates-1)))

idealVect=[['elective','math','math','machinelearning','software','math','machinelearning','math','software','practicalComponent','math','elective','machinelearning','software','expelective'],
           ['math','math','math','machinelearning','software','math','software','math','machinelearning','expelective','elective','elective','machinelearning','software','practicalComponent']]


            
#Set up graph enviornment for agent 
startState='initial'
env = Env(G, idealVect,actionVectors,startState)

In [ ]:
def choose_first_action(state,stateUTSoFar,topicVector,goalVector,actionVectors,w2,delta,beta,units_vect):

  ideal=[1]*73
  
  chosen_action= ('initial' , 'STATS 263') 
  state_vect=['expelective']
  List=[]
 
  for i in range(len(goalVector)):
    distance=levenshteinDistanceDP(state_vect[0], goalVector[i][0])
    if distance==0:
      List.append(1)
    else:
      List.append(0)
  
  
  currStateUT =delta * statistics.mean(List) + beta * w2
  units_vect.append(course_units_dict[chosen_action[1]])
   
  topicVect1=addBinary(topicVector, feature_ds_dict[chosen_action[1]])


  return chosen_action , stateUTSoFar+currStateUT , topicVect1,units_vect

In [ ]:
import time

#Do learning. Episode here is session
import copy

# start=time.time()
for episode in range(total_episodes):
    print("####################Episode##################=",episode)
    states=course_list_ds[:]
    reward = 0

#math course weight
    w1=0.25
    #experimental elective course weight
    w2=0.01
    #software course weight
    w3=0.15
    #machine learning weight
    w4=0.42
    #practical elective weight
    w5=0.01
    #other electives
    w6=0.16
    #ideal composition weight
    delta=0.8
    beta=0.2

    units_vect=[]
    stateUTSoFar = 0 
    state1 = env.reset()
    topicVect=[0]*73
    course_vector=[]
    action1,stateUTSoFar,topicVect1,units_vect= choose_first_action(state1,stateUTSoFar,topicVect,idealVect, actionVectors,w2,delta,beta,units_vect)
    state1Vect = [] # Make it length of num of ideal features 
    course_vector.append(action1[1])
  
    stateList1=[[]]*len(idealVect)
    while True: 
     
        #Getting the next state 
        state2, state2Vect ,reward, stateList2,units_vect,done = env.step(state1,action1,state1Vect,stateList1,w1,w2,w3,w4,w5,w6,delta,beta,units_vect,course_vector)
        
        states.remove(state2)
        # print(state2Vect)
       
        #Choosing the next action 
        stateList3=copy.deepcopy(stateList2)
        action2,stateUTSoFar, topicVect2,units_vect,course_vector= choose_action(state2,stateUTSoFar,topicVect1,idealVect,dataScience, states, state2Vect, stateList3,course_vector,w1,w2,w3,w4,w5,w6,delta,beta,units_vect) 
        
        if (action2==()):
          break
        else:
          course_vector.append(action2[1])
          
        #Updating the Q table
        update(state1, state2, action1, action2, reward,gamma,alpha) 
        
        state1 = state2 
        action1 = action2 
        state1Vect = state2Vect
        topicVect1=topicVect2
        stateList1=stateList2
        
        #If at the end of learning process, start another session
        if (done==True or states==[]): 
            break


In [ ]:
#Print Q Table as Dataframe
Qdf = pd.DataFrame(Q)
Qdf

In [ ]:
#Traverse the Q table

def getBestPath(QTable,startState):
  i = getKeysByValue(nodes,startState)
  loc = Qdf.iloc[i].idxmax()

  path =[]
  path.append(loc)
  while len(path) != 15:

    posOfMax = Qdf.iloc[loc].idxmax()
    if ((posOfMax in path )==True or prereq1(path,posOfMax)==0):
      # print('posofmax_1=',posOfMax)
      #print('loc',loc)
      #print('path',path)
      posOfMax_2=Qdf.iloc[loc,~Qdf.columns.isin(path)].idxmax()
      # print('posOfMax_2=',posOfMax_2)
      path.append(posOfMax_2) 
      loc=posOfMax_2  
    else:
      path.append(posOfMax)
      loc=posOfMax 
  return path

In [ ]:
print("Best sequence of courses to get a master degree in data science is: \n")
Seq=getBestPath(Qdf,startState)

Sequence={}
for i in Seq:
  Sequence[nodes[i]]=data_science_dict[nodes[i]]


for key in Sequence:
  if (Sequence[key]==[1,0,0,0,0,0]):
    Sequence[key]="math"
  elif (Sequence[key]==[0,1,0,0,0,0]):
    Sequence[key]="expelective"
  elif (Sequence[key]==[0,0,1,0,0,0]):
    Sequence[key]="software"
  elif (Sequence[key]==[0,0,0,1,0,0]):
    Sequence[key]="machinelearning"
  elif (Sequence[key]==[0,0,0,0,1,0]):
    Sequence[key]="practicalComponent"
  else:
    Sequence[key]="elective"

for i,j in Sequence.items():
  print('{} : {}'.format(i,j))

print('Number of recommended courses=',len(Sequence))

In [ ]:
domain=[]
for i in Sequence:
  domain.append(Sequence[i])
print(domain)

In [ ]:
#courses in each domain area
c=['elective','math','machinelearning','software','practicalComponent', 'expelective']
for i in c:
  print(i,domain.count(i))

In [ ]:
from collections import Counter

#define Jaccard Similarity function
def jaccard(list1, list2):
    intersection = list((Counter(list1) & Counter(list2)).elements())
    return len(intersection) 

In [ ]:
#Calculating the Score, details on this score are in our work

def score(List):
  result=[]

  for j in idealVect:
    score=0
    i=0
    while (i<=12):

      a=[]
      b=[]
      for k in range(i,i+5):
        a.append(j[k])
        b.append(List[k])
      # print('a=',a)
      # print('b=',b)
      # print(jaccard(a,b))
      score+=jaccard(a,b)
      i+=5
    result.append(score)
    # print(result)
  return max(result)

print('score=',score(domain))

In [ ]:
#Check the prerequisites condition
''' if all 1's means all prerequisites condition are satisfied '''

c=[]
for i in Sequence:
  c.append(i)
for i in Sequence:
  print(i,prereq(c,i))